### InkToFont Dataset

### import

In [77]:
import numpy as np
from PIL import Image, ImageFont, ImageDraw

### 글자를 정중앙에 작성하는 과정

In [78]:
def centerize_letter(letter, white, font):
  image = Image.fromarray(white, mode='L')
  draw = ImageDraw.Draw(image)
  draw.text((0, 0), letter, (0), font=font)

  array = np.array(image)
  left, right, up, down = 0, 0, 0, 0

  for row in range(128):
    if not (array[row] == 255).all():
      up = row
      break

  for row in range(127, -1, -1):
    if not (array[row] == 255).all():
      down = row
      break

  for col in range(128):
    if not (array[:, col] == 255).all():
      left = col
      break

  for col in range(127, -1, -1):
    if not (array[:, col]).all():
      right = col
      break

  dr = 64 - (up +  down) / 2
  dc = 64 - (left + right) / 2

  image = Image.fromarray(white, mode='L')
  draw = ImageDraw.Draw(image)
  draw.text((dc, dr), letter, (0), font=font)

  return image

### 모든 한글을 순서대로 구하는 함수

In [79]:
def nextKorLetterFrom(letter):
    lastLetterInt = 15572643

    if not letter:
        return '가'

    a = letter
    b = a.encode('utf8')
    c = int(b.hex(), 16)

    if c == lastLetterInt:
        return False

    d = hex(c + 1)
    e = bytearray.fromhex(d[2:])

    flag = True
    while flag:
        try:
            r = e.decode('utf-8')
            flag = False
        except UnicodeDecodeError:
            c = c + 1
            d = hex(c)
            e = bytearray.fromhex(d[2:])

    return e.decode()

In [81]:
letter = ''
white = np.full((128, 128), 255, dtype=np.uint8)

source_font_path = '/content/NanumGothic.ttf'
source_font =  ImageFont.truetype(source_font_path, 100)
target_font_path = '/content/NanumWild.ttf'
target_font = ImageFont.truetype(target_font_path, 100)

while True:
  letter = nextKorLetterFrom(letter)
  if not letter:
    break

  source_image = centerize_letter(letter, white, source_font)
  target_image = centerize_letter(letter, white, target_font)
  dst = Image.new('L', (256, 128))
  dst.paste(source_image, (0, 0))
  dst.paste(target_image, (128, 0))
  dst.save(f'dataset/{letter}.png')